In [ ]:
import pandas as pd
from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
from typing import List, Dict
from datetime import datetime
import sys
import os
import seaborn as sns
%matplotlib inline
import re

In [ ]:
notebook_dir = os.getcwd()
project_root = os.path.join(notebook_dir, '..','..','..','..','..','..','..','..')
sys.path.append(os.path.abspath(project_root))

print("notebook dir:", notebook_dir)
print("project root:", project_root)
print("absolute project root:", os.path.abspath(project_root))
print("notebook dir:", sys.path)

In [ ]:
notebook_dir = os.getcwd()
project_root = os.path.join(notebook_dir, '..')
sys.path.append(os.path.abspath(project_root))

sys.path.append(r"")

from utils.logger_config import get_sga_logger
 
logger = get_sga_logger()

In [ ]:

def log_exceptions(func):
    """
    Decorator to log exceptions in a function using the shared 'logger'.
    It will also re-raise the exception so that the caller can handle it
    appropriately (e.g., fail fast or continue).
    """
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as exc:
            logger.error(
                f"Error in function '{func.__name__}': {exc}",
                exc_info=True
            )
            # Optionally, decide whether to re-raise or swallow the exception.
            # Usually best practice is to re-raise so the pipeline can decide what to do:
            raise
    return wrapper



In [ ]:
def cut_decimal_part(df, column):
    """
    Converts a DataFrame column from float (or numeric string) to a string
    by removing the decimal part (i.e. converting 13.5 to "13", 12.0 to "12").
    Non-numeric values are converted to NaN and then to an empty string.
    """
    df[column] = pd.to_numeric(df[column], errors='coerce')

    df[column] = df[column].apply(lambda x: str(int(x)) if pd.notnull(x) else '')
    
    return df

In [ ]:
def float_to_hhmm(hours_float):
    hours = int(hours_float)
    minutes = int(round((hours_float - hours)*60))
    return f"{hours}:{minutes:02d}"

In [ ]:
def seconds_to_hhmm(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    return f"{hours}:{minutes:02d}"

In [ ]:
def handle_null_values(df, fill_str="", fill_float=0.0, fill_datetime=""):
    """
    Fill null values in DataFrame columns based on data type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        fill_str (str): Value to replace nulls in object/string columns. Default is "".
        fill_float (float): Value to replace nulls in float columns. Default is 0.0.
        fill_datetime: Value to replace nulls in datetime columns. 
                       Default is "", but you can also pass a default datetime.
    
    Returns:
        pd.DataFrame: The DataFrame with nulls handled.
    """

    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        df[col] = df[col].fillna(fill_str).astype(str)
    

    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].fillna(fill_float)
        

    datetime_cols = df.select_dtypes(include=['datetime64[ns]']) 
    for col in datetime_cols:
        df[col] = df[col].fillna(fill_datetime)
        
    return df

In [ ]:

def resolve_clock_stop_overlaps(clock_stops: List[Dict]) -> List[Dict]:
    """
    Eliminate overlaps in clock stops (paradas de reloj) by nro_incidencia.

    Args:   
        clock_stops: List of clock stops with 'start' 'end' datetime and 'nro_incidencia'

    Returns:
        List of non-overlapping clock stops
            
    """
    if not clock_stops:
        return []
    
    incidents = {}
    for stop in clock_stops:
        nro_incidencia = stop.get('nro_incidencia', 'unknown')
        if nro_incidencia not in incidents:
            incidents[nro_incidencia] = []
        incidents[nro_incidencia].append(stop)

    
    resolved_all = []   

    for nro_incidencia, incident_stops in incidents.items():
        sorted_stops = sorted(incident_stops, key=lambda x: x['start'])

        for i, stop in enumerate(sorted_stops):
            if pd.isna(stop['end']):
                if i < len(sorted_stops) - 1 and not pd.isna(sorted_stops[i+1]['start']):
                    stop['end'] = sorted_stops[i+1]['start']
                else:
                    logger.warning(f"Removing stop with missing end date for nro_incidencia {nro_incidencia}")
                    continue
        
        valid_stops = [stop for stop in sorted_stops if not pd.isna(stop['end'])]

        if not valid_stops:
            continue

        resolved_stops = [valid_stops[0]]

        for current_stop in valid_stops[1:]:
            last_resolved = resolved_stops[-1]

            if current_stop['start'] <= last_resolved['end']:
                last_resolved['end'] = max(last_resolved['end'], current_stop['end'])
            else:
                resolved_stops.append(current_stop)

        resolved_all.extend(resolved_stops)

    return resolved_all

@log_exceptions
def calculate_total_clock_stop_minutes(nro_incidencia:str, interruption_start: datetime, interruption_end: datetime, df_sga_paradas: pd.DataFrame) -> float:
    """
    Calculate the total clock minutes for a ticket, considering constraints.

    Args:
        nro_incidencia: The ticket identifier
        interrupcion_inicio: Start time of the interruption from REPORTE DINAMICO 335 
        interrupcion_fin: End time of the interruption from REPORTE DINAMICO 335 
    
    Returns:
        Total clock stop minutes
    
    """   
    df_sga_paradas['nro_incidencia'] = df_sga_paradas['nro_incidencia'].astype(str)
    nro_incidencia_stops = df_sga_paradas[df_sga_paradas['nro_incidencia'] == nro_incidencia].copy()

    if nro_incidencia_stops.empty:
        logger.info(f"No clock stops found for incident {nro_incidencia}")
        return 0.0
    
    clock_stops = []

    for _, stop in nro_incidencia_stops.iterrows():
        start_date = stop.get('startdate')
        end_date = stop.get('enddate')

        if pd.isna(start_date):
            logger.warning(f"Skipping record with missing start date for incident {nro_incidencia}")
            continue

        if start_date < interruption_start:
            logger.info(f"Adjusting start time to interruption en for incident {nro_incidencia}")
            start_date = interruption_start

        if not pd.isna(end_date):
            if end_date > interruption_end:
                logger.info(f"Adjusting end time to interruption en for incident {nro_incidencia}")
                end_date = interruption_end

            if start_date < end_date:
                clock_stops.append({
                    'start': start_date,
                    'end': end_date,
                    'nro_incidencia': nro_incidencia
                })
        else:
            clock_stops.append({
                'start': start_date,
                'end': end_date,
                'nro_incidencia': nro_incidencia
            })
    resolved_stops = resolve_clock_stop_overlaps(clock_stops)

    total_minutes = sum(
        (stop['end'] - stop['start']).total_seconds() / 60
        for stop in resolved_stops
        if not pd.isna(stop['end']) and not pd.isna(stop['start'])
    )
    return total_minutes



In [ ]:
def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [ ]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2_20250410_194426.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_04-04-2025_20250410_195338.xlsx"

In [ ]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)

In [ ]:
df_sga_dinamico_335['interrupcion_inicio'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_inicio'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['interrupcion_fin'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_fin'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_comunicacion_cliente'] = pd.to_datetime(df_sga_dinamico_335['fecha_comunicacion_cliente'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_generacion'] = pd.to_datetime(df_sga_dinamico_335['fecha_generacion'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fg_padre'] = pd.to_datetime(df_sga_dinamico_335['fg_padre'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['hora_sistema'] = pd.to_datetime(df_sga_dinamico_335['hora_sistema'], errors='coerce', dayfirst=True)
df_sga_dinamico_335["cid"] = df_sga_dinamico_335["cid"].astype(str).fillna("")
df_sga_dinamico_335['nro_incidencia'] = df_sga_dinamico_335['nro_incidencia'].astype(str)
df_sga_dinamico_335 = handle_null_values(df_sga_dinamico_335)
df_sga_dinamico_335["it_determinacion_de_la_causa"] = df_sga_dinamico_335["it_determinacion_de_la_causa"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335["tipo_caso"] = df_sga_dinamico_335["tipo_caso"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335["cid"] = df_sga_dinamico_335["cid"].astype(str).str.strip().fillna('No disponible')
df_sga_dinamico_335 = cut_decimal_part(df_sga_dinamico_335, 'codincidencepadre')

df_sga_dinamico_380['startdate'] = pd.to_datetime(df_sga_dinamico_380['startdate'],  errors='coerce', dayfirst=True)
df_sga_dinamico_380['enddate'] = pd.to_datetime(df_sga_dinamico_380['enddate'],  errors='coerce', dayfirst=True)
df_sga_dinamico_380 = handle_null_values(df_sga_dinamico_380)

df_corte_excel = cut_decimal_part(df_corte_excel,'CUISMP')
#df_corte_excel = cut_decimal_part(df_corte_excel,'CODINCIDENCEPADRE')
df_corte_excel = handle_null_values(df_corte_excel)
df_corte_excel = df_corte_excel.rename(columns={'TICKET':'nro_incidencia'})
df_corte_excel['nro_incidencia'] = df_corte_excel['nro_incidencia'].astype(str)
df_corte_excel['DF'] = df_corte_excel['DF'].astype(str).str.strip().fillna('No disponible').str.lower()
df_corte_excel['CUISMP'] = df_corte_excel['CUISMP'].astype(str).str.strip().fillna('No disponible')
df_corte_excel['DETERMINACIÓN DE LA CAUSA'] = df_corte_excel['DETERMINACIÓN DE LA CAUSA'].astype(str).str.strip().fillna("No disponible")
df_corte_excel['TIPO CASO'] = df_corte_excel['TIPO CASO'].astype(str).str.strip().fillna("No disponible")
df_corte_excel['CID'] = df_corte_excel['CID'].astype(str).str.strip().fillna("No disponible")

df_cid_cuismp_sharepoint = cut_decimal_part(df_cid_cuismp_sharepoint, 'CUISMP')
df_cid_cuismp_sharepoint = df_cid_cuismp_sharepoint.rename(columns={"CID":"cid"})
df_cid_cuismp_sharepoint["cid"] = df_cid_cuismp_sharepoint["cid"].astype(str).fillna("")
df_cid_cuismp_sharepoint["Distrito Fiscal"] = df_cid_cuismp_sharepoint["Distrito Fiscal"].astype(str).str.strip().fillna('No disponible').str.lower()
df_cid_cuismp_sharepoint["CUISMP"] = df_cid_cuismp_sharepoint["CUISMP"].astype(str).str.strip().fillna('No disponible')

In [ ]:
def merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel: pd.DataFrame, 
        df_sga_dinamico_335: pd.DataFrame,
        df_cid_cuismp_sharepoint: pd.DataFrame,
        df_sga_dinamico_380: pd.DataFrame,
        match_type:str
    ) -> pd.DataFrame:
        """
        Common merge function for Objective 1.

        Merges:
          - corte-excel  with sga_dinamico_335 on 'nro_incidencia'

        Returns a merged DataFrame with common columns needed.
        """

        merged_sga335_excel = pd.merge(
            df_corte_excel,
            df_sga_dinamico_335,
            on='nro_incidencia',
            how='left',
            indicator=True,
            suffixes=('_corte_excel', '_sga_dinamico_335')
        )

        merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp = pd.merge(
        merged_sga335_excel,
        df_cid_cuismp_sharepoint,
        on='cid',
        how='left',
        suffixes=('_sga_dinamico_335_excel_matched', '_sharepoint_cid_cuismp')
        )

        merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp['sum_paradas'] = merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp.apply(
            lambda r: calculate_total_clock_stop_minutes(
                nro_incidencia = r["nro_incidencia"],
                interruption_start = r["interrupcion_inicio"],
                interruption_end = r["interrupcion_fin"],
                df_sga_paradas = df_sga_dinamico_380
            ),
            axis= 1
        )

        matched_rows = merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp[merge_sga_335_corte_excel_matched_with_sharepoint_cid_cuismp['_merge'] == match_type]

        return matched_rows

In [ ]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
df_unmatched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
    df_corte_excel,
    df_sga_dinamico_335,
    df_cid_cuismp_sharepoint,
    df_sga_dinamico_380,
    'left_only'
    )


In [ ]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380.head(1)

In [ ]:
info =  get_dataframe_summary(df_matched_corte_sga335_Sharepoint_cuismp_sga380)
info


In [ ]:

@log_exceptions
def validation_medidas_correctivas(merged_df: pd.DataFrame) -> pd.DataFrame:
    """
    Validation the column medidas correctivas y o medidas tomadas, se debe obtener
    the first and the last date from paragraph, excluding the two last lines if exists 
    dates called fecha hora inicio and fecha hora fin
    """

    df = merged_df.copy()

    df['fechas_parrafos_coinciden'] = True
    df['fechas_parrafos_match_columns_first_date'] = True
    df['fechas_parrafos_match_columns_last_date'] = True
    df['no_errores_ortograficos'] = True

    def extract_dates(row):
        medidas_text = row['MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS']

        default = {
            'first_date':None,
            'last_date': None,
            'fecha_inicio': None,
            'fecha_fin': None,
            'last_date': None,
            'spelling_errors': False
        }

        if not isinstance(medidas_text, str) or not medidas_text.strip():
            return default
        
        lines = medidas_text.split('\n')
        paragraph_lines = []
        fecha_inicio_line = ""
        fecha_fin_line = ""

        if len(lines) >=2:
            if lines[-2].strip().startswith("Fecha y hora inicio"):
                fecha_inicio_line = lines[-2]
                paragraph_lines = lines[:-2]
            if lines[-1].strip().startswith("Fecha y hora fin"):
                fecha_fin_line = lines[-1]
                if not paragraph_lines:
                    paragraph_lines = lines[:-1]

        if not paragraph_lines:
            paragraph_lines = lines

        paragraph = '\n'.join(paragraph_lines)

        import re

        datepattern_in_paragraph = r'(?:el )?(?:dia)?(\d{1,2}/\d{1,2}/\d{4})(?:,)? (?:a las)?(\d{1,2}:\d{2})(?:\s?horas)?'
        datapattern_last_2_lines = r'\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2})'
        dates_in_paragraph = re.findall(datepattern_in_paragraph, paragraph)

        formatted_dates = []
        for date, time in dates_in_paragraph:
            date_parts = date.split('/')
            if len(date_parts) == 3:
                day = date_parts[0].zfill(2)
                month = date_parts[1].zfill(2)
                year = date_parts[2]
                date = f"{day}/{month}/{year}"

            time_parts = time.split(':')
            if len(time_parts) == 2:
                hour = time_parts[0].zfill(2)
                minute = time_parts[1].zfill(2)
                time = f"{hour}:{minute}"
            
            formatted_dates.append(f"{date} {time}")

        first_date = formatted_dates[0] if formatted_dates else None
        last_date = formatted_dates[-1] if len(formatted_dates) > 1 else first_date

        fecha_inicio = None
        if fecha_inicio_line:   
            inicio_match = re.search(datapattern_last_2_lines, fecha_inicio_line)
            if inicio_match:
                fecha_inicio = inicio_match.group(0)

        fecha_fin = None
        if fecha_fin_line:
            fin_match = re.search(datapattern_last_2_lines, fecha_fin_line)
            if fin_match:
                fecha_fin = fin_match.group(0)

        spelling_errors = False
        error_patterns = [
            r'inmendiatamente'
        ]

        for pattern in error_patterns:
            if re.search(pattern, paragraph, re.IGNORECASE):
                spelling_errors  = True
                break
        return{
            'first_date': first_date,
            'last_date': last_date,
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'spelling_errors': spelling_errors,
        }
    
    results = df.apply(extract_dates, axis=1)
    results_df = pd.DataFrame(results.tolist(), index=df.index)

    df = pd.concat([df,  results_df], axis=1)
    
    df['fechas_parrafos_match_columns_first_date'] = (
        df['first_date'] == df['FECHA Y HORA INICIO']
    )

    df['fechas_parrafos_match_columns_last_date'] = (
        df['last_date'] == df['FECHA Y HORA FIN']
    )

    df['no_errores_ortograficos'] = ~df['spelling_errors']

    df['validation_medidas_correctivas_ok'] = (
        df['fechas_parrafos_match_columns_first_date'] &
        df['fechas_parrafos_match_columns_last_date'] &
        df['no_errores_ortograficos'] 
    )

    df['Validation_OK'] = df['validation_medidas_correctivas_ok']
    df['fail_count'] = (~df['Validation_OK']).astype(int)
